# **Obtención y preparación de datos**

# OD15. Reindexación de Estructuras en Pandas

Creación de una copia de una estructura pandas -una serie o un dataframe- en base a un nuevo índice.

In [1]:
import numpy as np
import pandas as pd

## <font color='blue'>**Reindexación de series**</font>

El método básico para la reindexación de series es **pandas.Series.reindex**. Este método devuelve una copia de una serie basándose en el índice modificado de la serie original.

In [2]:
s = pd.Series([1, 2, 3, 4, 5], index = ["a", "c", "f", "g", "j"])
s

a    1
c    2
f    3
g    4
j    5
dtype: int64

Se trata de una serie cuyas etiquetas son letras no consecutivas. Es posible generar una copia reindexada de esta serie de la siguiente forma:



In [3]:
r = s.reindex(["g", "c", "a", "j", "f"]) #reindex :cambia la indexación 
r

g    4
c    2
a    1
j    5
f    3
dtype: int64

El primer argumento siempre es el nuevo índice. En el caso anterior se trata de una versión desordenada del índice original, por lo que la serie generada es también una versión desordenada de la original.

Si el nuevo índice es un subconjunto del original, la serie generada no contendrá todos los valores de la serie de la que se patió, tan solo los incluidos en el nuevo índice. En el siguiente ejemplo, el nuevo índice no incluye la etiqueta "a" por lo que la serie generada no incluye el valor correspondiente (1):

In [4]:
r = s.reindex(["g", "c", "j", "f"]) #reindex descarta los que no son declarados
r

g    4
c    2
j    5
f    3
dtype: int64

Por el contrario, si en el nuevo índice se incluyen etiquetas no incluidas en el índice original, la nueva serie incluirá dicha etiqueta pero el valor asignado a ella recibe el valor por defecto NaN. En este próximo ejemplo incluimos la etiqueta "e" (no presente en el índice original) en el nuevo índice:

In [5]:
r = s.reindex(["g", "c", "e", "a", "j", "f"]) #si en el reindex se ingresan index que no estaban, estos se ingresan con valor NaN
r

g    4.0
c    2.0
e    NaN
a    1.0
j    5.0
f    3.0
dtype: float64

Este valor de relleno ("NaN") es personalizable usando el parámetro **fill_value**. Si repetimos las instrucciones anteriores especificando que el valor de relleno sea, por ejemplo, 0:

In [6]:
r = s.reindex(["g", "c", "e", "a", "j", "f"], fill_value = 0) #fill_value: rellena los valores naN
r

g    4
c    2
e    0
a    1
j    5
f    3
dtype: int64

Una alternativa a fijar el valor de relleno por defecto es aplicar "lógica de relleno", rellenando los valores inexistentes con otro valor que sí exista. Tenemos tres opciones...

## <font color='blue'>**Forward fill**</font>

La primera opción consiste en rellenar los valores inexistentes "hacia adelante", haciendo que los valores existentes rellenen los valores inexistentes que los sigan. O, en otras palabras, rellenar los valores inexistentes con el primer valor existente que los precedan:

In [7]:
s = pd.Series([1, 2, 3, 4, 5], index = ["a", "c", "f", "g", "j"])
s

a    1
c    2
f    3
g    4
j    5
dtype: int64

In [8]:
r = s.reindex(["g", "c", "e", "a", "j", "f"], method = "ffill") #se reemplaza con el último valor válido anterior en orden (c)
r

g    4
c    2
e    2
a    1
j    5
f    3
dtype: int64

<font color="orange">Method{None, ‘backfill’/’bfill’, ‘pad’/’ffill’, ‘nearest’}
Method to use for filling holes in reindexed DataFrame. Please note: this is only applicable to DataFrames/Series with a monotonically increasing/decreasing index.</font>

<font color="orange">None (default): don’t fill gaps</font>


<font color="orange">pad / ffill: Propagate last valid observation forward to next valid.</font>

<font color="orange">backfill / bfill: Use next valid observation to fill gap.</font>

<font color="orange">nearest: Use nearest valid observations to fill gap.</font>

En este caso, el valor correspondiente a la etiqueta "e" se rellena con el valor de la etiqueta anterior "c". Pero no la anterior en el nuevo índice, sino la anterior en el índice original. Veámoslos con otro ejemplo:

In [9]:
r = s.reindex(["g", "c", "m", "a", "j", "f"], method = "ffill") #idem al anterior, el último index válido antes de m es j 
r

g    4
c    2
m    5
a    1
j    5
f    3
dtype: int64

Ahora, la etiqueta nueva es "m", siendo precedida en el índice original por la "j" (si se ordenan alfabéticamente), por lo que el valor que recibe r["m"] es el que tenía r["j"]: 5.

## <font color='blue'>**Backward fill**</font>

En este otro caso, los valores inexistentes se rellenan "hacia atrás", con el primer valor existente que los siga.

In [10]:
s = pd.Series([1, 2, 3, 4, 5], index = ["a", "c", "f", "g", "j"])
s

a    1
c    2
f    3
g    4
j    5
dtype: int64

In [11]:
r = s.reindex(["g", "c", "e", "a", "j", "f"], method = "bfill") #se reemplaza por el posterior (en este caso es f)
r

g    4
c    2
e    3
a    1
j    5
f    3
dtype: int64

Nuevamente, la etiqueta no existente en el índice original es "e", y el valor que se le asigna es el correspondiente a la etiqueta que seguía a "e" en dicho índice (si se ordenan alfabéticamente): "f". Por lo tanto, se asigna a r["e"] el valor de r["f"]: 3.

## <font color='blue'>**Nearest (el más cercano)**</font>

La tercera opción asigna a cada valor desconocido el valor más próximo en la serie original. Para ver esta opción en funcionamiento necesitamos partir de una serie cuyo índice sea numérico (la operación "sustracción" en la que se basa esta tercera opción no está soportada entre cadenas de texto).

In [12]:
s = pd.Series([100, 200, 300, 400, 500], index = [10, 20, 30, 40, 50])
s

10    100
20    200
30    300
40    400
50    500
dtype: int64

Una serie cuyo índice está formado por múltiplos de 10. Generemos ahora una copia del mismo con el índice [20, 40, 19] aplicando como método de relleno "nearest".

In [13]:
r = s.reindex([20, 40, 19], method = "nearest") #el index que encuentre más cercano al que no tiene valor
r

20    200
40    400
19    200
dtype: int64

El método ha incluido el índice 19 y le ha asignado el valor del índice más próximo (20), es decir, el valor de s[20] (200).

In [14]:
r = s.reindex([20, 40, 11], method = "nearest")
r

20    200
40    400
11    100
dtype: int64

En este caso, el índice más próximo es 10, y el valor asignado es, por lo tanto, s[10]: 100.

## <font color='blue'>**Reindexación de dataframes**</font>

El método **pandas.DataFrame.reindex** ofrece una funcionalidad semejante a la disponible para series con la particularidad de que, en este caso, podemos reindexar por filas y/o por columnas. Por defecto, este método acepta una secuencia de etiquetas que determinarán qué filas se incluyen y en qué orden (es decir, por defecto la reindexación se aplica al eje 0).

In [15]:
df = pd.DataFrame(np.arange(15).reshape([5, 3]),
                  index = ["a", "b", "c", "d", "e"],
                  columns = ["A", "B", "C"])
df

,A,B,C
a,0,1,2
b,3,4,5
c,6,7,8
d,9,10,11
e,12,13,14


In [16]:
df.reindex(["d", "b"]) #reindex puede ser usado como filtro de filas también

,A,B,C
d,9,10,11
b,3,4,5


En este ejemplo, partimos de un dataframe cuyo índice de filas tiene las etiquetas "a", "b", "c", "d" y "e", y hemos indicado como nuevo índice las etiquetas "d" y "b" (en este orden), y son estas filas (en ese orden) las que se devuelven como resultado.

Este método permite especificar las etiquetas de filas como hemos visto, pasándoselas al método como primer argumento, o con el parámetro **index**.

In [17]:
df.reindex(index = ["d", "b"]) #lo mismo, aunque se específica index como filas

,A,B,C
d,9,10,11
b,3,4,5


El parámetro **columns**, por su parte, permite especificar el nuevo índice de columnas:



In [18]:
df.reindex(columns = ["A", "C"]) #lo mismo, en este caso para columnas

,A,C
a,0,2
b,3,5
c,6,8
d,9,11
e,12,14


Si utilizamos ambos parámetros al mismo tiempo, imponemos simultáneamente el nuevo índice para filas y columnas.

In [19]:
df.reindex(index = ["a", "c", "f"], columns = ["A", "D", "C"]) #se pueden incluir tanto para filas y columnas

,A,D,C
a,0.0,NaN,2.0
c,6.0,NaN,8.0
f,NaN,NaN,NaN


Podemos asignar a los valores inexistentes un valor concreto usando el parámetro **fill_value**, o podemos aplicar "lógica de relleno" con el parámetro **method**, permitiéndonos rellenar los valores inexistentes hacia adelante o hacia atrás.

Y, por supuesto, si los nuevos índices contienen los mismos elementos que los índices originales pero en otro orden, el resultado del método será equivalente al original ordenado según el nuevo criterio.

In [20]:
df.reindex(index = ["a", "c", "b", "e", "d"], columns = ["B", "C", "A"])

,B,C,A
a,1,2,0
c,7,8,6
b,4,5,3
e,13,14,12
d,10,11,9


## <font color='blue'>**Método set_index**</font>

El método **pandas.DataFrame.set_index** fija una columna del dataframe como índice, descartando el índice existente.

In [21]:
df = pd.DataFrame({
    "año": [2016, 2017, 2018],
    "mes": ["ene", "sep", "jun"],
    "ventas": [87, 34, 112]
})
df

,año,mes,ventas
0,2016,ene,87
1,2017,sep,34
2,2018,jun,112


Vemos que se ha asignado un índice automático. Si ejecutamos el método set_index indicando como argumento el campo "mes".

In [22]:
df.set_index("mes") #set_index cambia el index

,año,ventas
mes,,
ene,2016,87
sep,2017,34
jun,2018,112


Se fija dicha columna como índice y se elimina del conjunto de características. Aunque esta eliminación es el comportamiento por defecto, podemos controlarlo con el parámetro drop.

In [23]:
df.set_index("mes", drop = False) #deja la columna tanto como index y columna

,año,mes,ventas
mes,,,
ene,2016,ene,87
sep,2017,sep,34
jun,2018,jun,112


## <font color="red">Experimentos reindexación:</font>

## <font color="red">Índices enteros:</font>

In [24]:
s1 = pd.Series([i for i in range(5)])
s1

0    0
1    1
2    2
3    3
4    4
dtype: int64

- <font color="red">Acepta índices enteros.</font>
- <font color="red">La reindexación puede repetir los índices (es parecido a sample)</font>


In [25]:
s1.reindex(np.random.randint(0,5,5))

2    2
1    1
0    0
1    1
2    2
dtype: int64

## <font color="red">Índices numéricos:</font>

In [26]:
s2 = pd.Series([i for i in range(5)], index = [3, 5.2, 2.4, 4, 0])
s2

3.0    0
5.2    1
2.4    2
4.0    3
0.0    4
dtype: int64

<font color="red">Deja los números tal como se definieron, aunque transforma el tipo a float.</font>

In [27]:
s2.reindex([i for i in range(5)]) #reindexa a números enteros

0    4.0
1    NaN
2    NaN
3    0.0
4    3.0
dtype: float64

In [28]:
[type(i) for i in s2.index] #index queda como float

[float, float, float, float, float]

<font color="red">Genera un <font color='*red'>**Error**</font> al intentar rellenar si los indices son tipo ***float***.</font>

In [30]:
s2.reindex([0.0, 2.0, 6.2, 8.4], method='bfill')

ValueError: ignored

In [31]:
s2.reindex([0.0, 2.0, 6.2, 8.4], method='nearest')

ValueError: ignored

## <font color="red">Índices letras +  números:</font>

In [32]:
s3 = pd.Series([i for i in range(7)], index = [0, 1.3, 'two', 3.14, 'cuatro', 'c', 6])
s3

0         0
1.3       1
two       2
3.14      3
cuatro    4
c         5
6         6
dtype: int64

<font color="red">Los índices conservan el tipo se hay una combinación de letras y números.</font>

In [33]:
s3.reindex(['0', '1.3', 'cuatro'])

0         NaN
1.3       NaN
cuatro    4.0
dtype: float64

In [34]:
[type(i) for i in s3.index]

[int, float, str, float, str, str, int]

<font color="red">Genera un <font color='*red'>**Error**</font> al intentar rellenar si los indices son de varios tipos.</font>

In [35]:
s3.reindex([0, 1.3, 'cuatro'], method='bfill') 
#en estos casos se encesita una ordenación de index, al tener distintos tipos, no puede encontrar una forma de ordenación, retornando error

ValueError: ignored

##<font color="darkorange">Índices tipo string:</font>

In [36]:
s4 = pd.Series([1, 2, 3, 4, 5], index = ["avion", "chao", "f", "gato", "ja"])
s4

avion    1
chao     2
f        3
gato     4
ja       5
dtype: int64

<font color="red">Al considerar letras y palabras, el método busca la primera letra del índice para asociar. Además, de lo que se muestra más abajo lo hace de acuerdo al orden del alfabeto (completó la *i*, pero no la *k*).</font>

In [37]:
s4.reindex(['a', 'f', 'g', 'h', 'i', 'z', 'k'], method='bfill')

a    1.0
f    3.0
g    4.0
h    5.0
i    5.0
z    NaN
k    NaN
dtype: float64

##<font color="red">Índices y columnas de varios tipos (dataframes):</font>

In [38]:
df1 = pd.DataFrame(np.arange(15).reshape([5, 3]),
                  index = [i*0.5 for i in range(5)],
                  columns = ["a", "casa", "perro"])
df1

,a,casa,perro
0.0,0,1,2
0.5,3,4,5
1.0,6,7,8
1.5,9,10,11
2.0,12,13,14


<font color="red">Mantiene los tipos</font>

In [39]:
[
    [type(i) for i in df1.index],
    [type(i) for i in df1.columns]
]

[[float, float, float, float, float], [str, str, str]]

<font color="red">En este caso busca la palabra completa (no por la primera letra).</font>

In [40]:
df1.reindex(index = [0, 1.5, 2, 1, 6], columns = ["a", "perro", "c"])

,a,perro,c
0.0,0.0,2.0,NaN
1.5,9.0,11.0,NaN
2.0,12.0,14.0,NaN
1.0,6.0,8.0,NaN
6.0,NaN,NaN,NaN
